In [20]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import pandas as pd

In [2]:
def get_credentials(service_key_file, scopes):
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secrets.json', scopes=scopes)
    
    return credentials

In [5]:
credent = get_credentials(scopes)

In [30]:
def get_service_client(scopes, api_name, api_version, service_key_file=None, credentials=None):
    
    if credentials:
        service_client = build(api_name, api_version, credentials)
    
    else:
        credentials = ServiceAccountCredentials.from_json_keyfile_name(service_key_file, scopes=scopes)
        print(credentials, api_name, api_version)
        service_client = build(api_name, api_version, credentials)
    
    return service_client

In [29]:
reporting_client = get_service_client(scopes, 'analyticsreporting', 'v4', service_key_file='client_secrets.json')

<oauth2client.service_account.ServiceAccountCredentials object at 0x00000124BB131588> analyticsreporting v4


AttributeError: 'ServiceAccountCredentials' object has no attribute 'request'

In [31]:
# Turn this into a function that returns the reporting_client

#Create the credentials object
service_credentials_file = 'client_secrets.json'
scopes = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/analytics.edit']
credentials = ServiceAccountCredentials.from_json_keyfile_name(service_credentials_file, scopes=scopes)

#Build a service object
api_name = 'analyticsreporting'
api_version = 'v4'
reporting_client = build(api_name, api_version, credentials=credentials)

In [32]:
# Turn into a report getting function

view_id = '95100800'

report_output = reporting_client.reports().batchGet(
    body={
    'reportRequests': [
        {
          'viewId': view_id,
          'dateRanges': [{'startDate': '28daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}],
          'dimensions': [{'name': 'ga:country'}]
        }]
      }
    ).execute()

In [4]:
type(report_output)

dict

In [33]:
report_output['reports']

[{'columnHeader': {'dimensions': ['ga:country'],
   'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
      'type': 'INTEGER'}]}},
  'data': {'maximums': [{'values': ['4']}],
   'minimums': [{'values': ['1']}],
   'rowCount': 3,
   'rows': [{'dimensions': ['Italy'], 'metrics': [{'values': ['1']}]},
    {'dimensions': ['Philippines'], 'metrics': [{'values': ['1']}]},
    {'dimensions': ['United States'], 'metrics': [{'values': ['4']}]}],
   'totals': [{'values': ['6']}]}}]

In [15]:
def print_response(response):
  
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for row in report.get('data', {}).get('rows', []):
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])
        
        for header, dimension in zip(dimensionHeaders, dimensions):
            print(header + ': ' + dimension)
        
        for i, values in enumerate(dateRangeValues):
            print('Date range: ' + str(i))
            
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                print(metricHeader.get('name') + ': ' + value)



In [17]:
print_response(report_output)

ga:country: Italy
Date range: 0
ga:sessions: 1
ga:country: Philippines
Date range: 0
ga:sessions: 1
ga:country: United States
Date range: 0
ga:sessions: 4


In [6]:
# Will need to modify this for my needs, temporarily borrowed
#Parse the response of API
def parse_response(report):

    """Parses and prints the Analytics Reporting API V4 response"""
    #Initialize results, in list format because two dataframes might return
    result_list = []

    #Initialize empty data container for the two dateranges (if there are two that is)
    data_csv = []
    data_csv2 = []

    #Initialize header rows
    header_row = []

    #Get column headers, metric headers, and dimension headers.
    columnHeader = report.get('columnHeader', {})
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    dimensionHeaders = columnHeader.get('dimensions', [])

    #Combine all of those headers into the header_row, which is in a list format
    for dheader in dimensionHeaders:
        header_row.append(dheader)
    for mheader in metricHeaders:
        header_row.append(mheader['name'])

    #Get data from each of the rows, and append them into a list
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        row_temp = []
        dimensions = row.get('dimensions', [])
        metrics = row.get('metrics', [])
        for d in dimensions:
            row_temp.append(d)
        for m in metrics[0]['values']:
            row_temp.append(m)
            data_csv.append(row_temp)

        #In case of a second date range, do the same thing for the second request
        if len(metrics) == 2:
            row_temp2 = []
            for d in dimensions:
                row_temp2.append(d)
            for m in metrics[1]['values']:
                row_temp2.append(m)
            data_csv2.append(row_temp2)

    #Putting those list formats into pandas dataframe, and append them into the final result
    result_df = pd.DataFrame(data_csv, columns=header_row)
    result_list.append(result_df)
    if data_csv2 != []:
        result_list.append(pandas.DataFrame(data_csv2, columns=header_row))

    return result_list

In [7]:
result = parse_response(report_output)

In [9]:
result

[Empty DataFrame
 Columns: []
 Index: []]